# Prediction for Control

### Import Packages

In [ ]:
using ForwardDiff
using DelimitedFiles
using LinearAlgebra
using MathOptInterface
using Parametron
using SparseArrays
using Plots
using OSQP
using PyCall
push!(PyVector(pyimport("sys")."path"), "../scripts")
python_utils = pyimport("python_utils")
np = pyimport("numpy");

### Include Julia Scripts

In [ ]:
include("../scripts/path_handling.jl")
include("../scripts/utils.jl")
include("../scripts/costs.jl")
include("../scripts/dynamics.jl")
include("../scripts/interface.jl")
include("../scripts/optimization_constraints.jl")
include("../scripts/mpc.jl")
include("../scripts/visualization.jl");

### Load Data

In [ ]:
# load data
env = python_utils.load_data_set("../../experiments/processed/nuScenes_val_full_doubled.pkl");

### Load Model and Settings

In [ ]:
# load model
model_path = "../../experiments/nuScenes/models/models_21_Jul_2020_10_25_10_full_zeroRrows_batch8_double_fixed_a_norm"
mats, hyperparams = load_model(model_path, env, ts=11)

# model prediction settings
num_modes = 3
pred_settings = PredictionSettings(mats, hyperparams, env, num_modes);

### Scene Selection

In [ ]:
# select scene
scene_num = 24 # corresponds to 23 in python indexing
scene = Scene(env, scene_num);

### Initialize and Construct MPC Problem

In [ ]:
if !isdir("data")
    mkdir("data")
end

# select time interval
first_ts = 3 
last_ts = scene.timesteps;

# get path data
x_coefs_var, y_coefs_var, breaks_var = load_splines()
path_obj = SplinePath(x_coefs_var, y_coefs_var, breaks_var)

# robot initial state
q0 = [scene.robot.x[first_ts], scene.robot.y[first_ts], scene.robot.θ[first_ts], scene.robot.v[first_ts], 0]
q0[5] = find_best_s(q0, path_obj, enable_global_search=true)

# MPC parameters, constraints and settings
control_limits_obj = ControlLimits(.7, -.7, 4., -5., 12., 0.)
dynamics_obj = DynamicsModel(4, 2, control_limits_obj)
vals_obj = MPCValues(path_obj, n_modes=num_modes, N=25, k_c=4, q0=q0, n_obs=length(scene.nodes))

# make first predictions for obstacle constraints
init_node_obstacles!(scene, vals_obj)
Aps, Bps, gps, q_pred0, nodes_present = predicted_dynamics(pred_settings, scene_num, first_ts)
u_pred = get_recorded_robot_controls(pred_settings, scene_num, first_ts)
q_pred = [predict_future_states(q_pred0, u_pred, Aps, Bps, gps, j) for j = 1:vals_obj.n_modes]
update_obstacles_from_predictions!(q_pred, nodes_present, vals_obj, scene)

# initial solution guess
qs, us = initial_guess(vals_obj);

In [ ]:
# construct problem
model, params, variables, mpc_values = construct_problem(dynamics_obj, vals_obj, scene, qs, us, verbose=false);

### Solve MPC

In [ ]:
x_traj = Array{Float64,1}()
y_traj = Array{Float64,1}()
θ_traj = Array{Float64,1}()
v_traj = Array{Float64,1}()
s_traj = Array{Float64,1}()

ω_traj = []
a_traj = []
vs_traj = []

x_pred = []
y_pred = []
θ_pred = []
v_pred = []
s_pred = []

ω_pred = []
a_pred = []
vs_pred = []

agent_states = []

ss =[0:0.01:138;]
spline_idces = find_spline_interval.(ss, (path_obj,))
x_path = spline_x.(ss, (path_obj,), spline_idces)
y_path = spline_y.(ss, (path_obj,), spline_idces)

n_iter_val = 3 # number of solve/relinearization iterations per timestep
    
qs = []
us = []

problem_infeasible = false

if !isdir("figs")
    mkdir("figs")
end

for ts = first_ts:last_ts
    println("--------------------------------------------")
    println("Starting iteration ", ts - first_ts + 1, "!")
    

    if ts == first_ts
        n_iter = 15
    else
        n_iter = n_iter_val
    end

    for j = 1:n_iter
        solve!(model)
        
        println("Termination Status: ", terminationstatus(model))
        println("Relinearization loop: ", j)

        output_qs = value.(model, variables.q)
        output_us = value.(model, variables.u)

        if (sum(isnan.(output_qs)) > 0) || (sum(isnan.(output_us)) > 0)
            println("Infeasible. Skipping to next iteration.")
            problem_infeasible = true
            break
        else
            qs = output_qs
            us = output_us
        end    

        relinearize!(mpc_values, qs, us)

        flush(stdout)
    end

    if problem_infeasible
        println("Coasting along previous solution.")
        flush(stdout)
        problem_infeasible = false
        qs, us = update_problem!(mpc_values, qs, us)

        push!(x_traj, qs[1,1])
        push!(y_traj, qs[2,1])
        push!(θ_traj, qs[3,1])
        push!(v_traj, qs[4,1])
        push!(s_traj, qs[5,1])

        push!(ω_traj, us[1,1])
        push!(a_traj, us[2,1])
        push!(vs_traj, us[3,1])

        push!(x_pred, qs[1,:])
        push!(y_pred, qs[2,:])
        push!(s_pred, qs[5,:])

        plot!(x_path, y_path, label="path", color=:blue, legend=:topright)
        scatter!([x_traj[ts]], [y_traj[ts]], marker=:dot, color=:black, ms=3, label="pos")
        scatter!(x_pred[ts], y_pred[ts], marker=:dot, color=:red, ms=1, label="pred", dpi=300)

        continue
    end

    push!(x_traj, qs[1,1])
    push!(y_traj, qs[2,1])
    push!(θ_traj, qs[3,1])
    push!(v_traj, qs[4,1])
    push!(s_traj, qs[5,1])

    push!(ω_traj, us[1,1])
    push!(a_traj, us[2,1])
    push!(vs_traj, us[3,1])

    push!(x_pred, qs[1,:])
    push!(y_pred, qs[2,:])
    push!(θ_pred, qs[3,:])
    push!(v_pred, qs[4,:])
    push!(s_pred, qs[5,:])
    
    push!(ω_pred, us[1,:])
    push!(a_pred, qs[2,:])
    push!(vs_pred, qs[3,:])

    # visualization
    plot()
    viz_path!(x_path, y_path)
    viz_solution!(qs, vals_obj)   
    viz_obstacle!(vals_obj)
    fig_settings!((x_path[1] - 100, x_path[end] + 100), (y_path[2] - 125, y_path[2] + 25))
    savefig("figs/" * string(ts - first_ts + 1) * ".png")
    
    # save data
    save_obstacle_data(ts - first_ts + 1, vals_obj)
    if (ts - first_ts + 1) == 11
        np.save("data/x_traj", x_traj)
        np.save("data/y_traj", y_traj)
        np.save("data/θ_traj", θ_traj)
        np.save("data/v_traj", v_traj)
        np.save("data/s_traj", s_traj)
        
        np.save("data/ω_traj", ω_traj)
        np.save("data/a_traj", a_traj)
        np.save("data/vs_traj", vs_traj)
        
        np.save("data/x_pred", x_pred)
        np.save("data/y_pred", y_pred)
        np.save("data/θ_pred", θ_pred)
        np.save("data/v_pred", v_pred)
        np.save("data/s_pred", s_pred)
        
        np.save("data/ω_pred", ω_pred)
        np.save("data/a_pred", a_pred)
        np.save("data/vs_pred", vs_pred)
    end
    
    
    pred_horizon = hyperparams.get("prediction_horizon")
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 1] = qs[1, 1:pred_horizon+1]  # x 
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 2] = qs[2, 1:pred_horizon+1]  # y
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 9] = qs[3, 1:pred_horizon+1]  # θ
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon, 10] = us[1, 1:pred_horizon]     # ω
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 11] = qs[4, 1:pred_horizon+1] # v
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon, 12] = us[2, 1:pred_horizon]     # a
    
    Aps, Bps, gps, q_pred0, nodes_present = predicted_dynamics(pred_settings, scene_num, ts+1)
    
    u_pred = us[1:2, 1:pred_horizon]' # use control from optimization solution
    q_pred = [predict_future_states(q_pred0, u_pred, Aps, Bps, gps, j) for j = 1:vals_obj.n_modes]
    
    update_obstacles_from_predictions!(q_pred, nodes_present, vals_obj, scene, iter=ts-first_ts+2)
    update_problem!(mpc_values, qs, us)
end